In [1]:
from pathlib import Path
import pandas as pd
from nuztf.fritz import fritz_api

In [2]:
# source_spec = "lris20240503_ZTF24aajztsf_o1.spec"
# source_spec = "ZTF18abxnvoz_DBSP_2024-08-13.txt"
# source_spec = "lris20240909_ZTF24aaxzkdq_k3.spec"

send_to_fritz = True

instrument_id = 7 # LRIS is 7

In [3]:
# path = Path.home().joinpath(f"Downloads/{source_spec}")
# path = Path.home().joinpath(f"Data/spectroscopy/LRIS_20240908/{source_spec}")


spec_dir = Path.home().joinpath(f"Data/spectroscopy/LRIS_20250522")

paths = list(spec_dir.glob("lris*.spec"))
# list(paths)

In [4]:
for path in paths:
    assert path.exists()

    # LRIS
    df = pd.read_table(
        path, sep="\s+", comment="#",
        names=["#wl", "flux", "sky_flux", "flux_unc", "xpixel", "ypixel", "response", "flag"]
    )

    # Fix for SNID
    df = df[df["flux"] > 0]
    outpath = path.parent / f"clean_{path.name}"
    print(f"Saving to {outpath}")
    df.to_csv(outpath, sep=" ", index=False)

    if send_to_fritz:
        # Upload to Fritz
        header = dict()
        with open(path, "r") as f:
            for line in f.readlines():
                if (line[0]=="#"):
                    try:
                        entry = line[1:].split("=")
                        key = entry[0].strip()
                        val = entry[1].split("/")[0].strip()
                        header[key] = val
                    except IndexError:
                        pass
        
        with open(path, "r") as f:
            text = f.read()
    
        source = header["OBJECT"].split("_")[0].replace("'", "")
        date = header["DATE"].replace("'", "").strip()
        
        payload = {
            "wave_column": 0,
            "flux_column": 1,
            "fluxerr_column": 3,
            "observed_at": date,
            "obj_id": source,
            "instrument_id": instrument_id,
            "ascii": text,
            "filename": path.name,
            "group_ids": [1]
        }
    
        res = fritz_api(
            method="GET", 
            endpoint_extension=f"api/sources/{source}/spectra",
        )
        existing_spectra = res.json()["data"]["spectra"]
        filenames = [x["original_file_filename"] for x in existing_spectra]
    
        if path.name in filenames:
            print(f"Spectrum {path.name} already uploaded.")
        else:
            print(f"Uploading spectrum {path.name}.")
            res = fritz_api(
                method="POST", 
                endpoint_extension="api/spectra/ascii",
                data=payload
            )
            res.raise_for_status()
    

Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF25aapxick.spec
Uploading spectrum lris20250523_ZTF25aapxick.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF18abqkfhj.spec
Uploading spectrum lris20250523_ZTF18abqkfhj.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lrisr20250523_ZTF25aaqkhyr.spec
Uploading spectrum lrisr20250523_ZTF25aaqkhyr.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF25aaqqfsk_k3.spec
Uploading spectrum lris20250523_ZTF25aaqqfsk_k3.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF24aaizowg.spec
Uploading spectrum lris20250523_ZTF24aaizowg.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF25aaplxiz.spec
Uploading spectrum lris20250523_ZTF25aaplxiz.spec.
Saving to /Users/robertstein/Data/spectroscopy/LRIS_20250522/clean_lris20250523_ZTF25aaqoagl_k2.sp

In [5]:
# download_dir = spec_dir = Path.home().joinpath("Downloads")

# path = download_dir / "ZTF24abfuwbm.csv"

In [6]:
# df = pd.read_csv(
#     path, comment="#",
#     names=["#wl", "flux", "flux_unc"]
# )

# # Fix for SNID
# df = df[df["flux"] > 0]
# outpath = path.parent / f"clean_{path.name}"
# print(f"Saving to {outpath}")
# df[["#wl", "flux"]].to_csv(outpath, sep=" ", index=False)